In [189]:
import pandas as pd

In [245]:
circuits = pd.read_csv('../data/circuits.csv')
constructor_results = pd.read_csv('../data/constructor_results.csv')
constructor_standings = pd.read_csv('../data/constructor_standings.csv')
constructors = pd.read_csv('../data/constructors.csv')
driver_standings = pd.read_csv('../data/driver_standings.csv')
drivers = pd.read_csv('../data/drivers.csv', parse_dates=[6])
lap_times = pd.read_csv('../data/lap_times.csv')
pit_stops = pd.read_csv('../data/pit_stops.csv')
qualifying = pd.read_csv('../data/qualifying.csv')
races = pd.read_csv('../data/races.csv', parse_dates=[5])
results = pd.read_csv('../data/results.csv')
seasons = pd.read_csv('../data/seasons.csv')
status = pd.read_csv('../data/status.csv')

In [246]:
result_with_dob = results.merge(drivers, left_on='driverId', right_on='driverId')
results_with_dob_date = result_with_dob.merge(races, on = 'raceId', how = 'left')

In [247]:
#results_with_dob_date['ageAtRace'] = pd.to_datetime(results_with_dob_date['date']) - pd.to_datetime(results_with_dob_date['dob'])
pd.to_datetime(results_with_dob_date['date'])
pd.to_datetime(results_with_dob_date['dob'])

0       1985-01-07
1       1985-01-07
2       1985-01-07
3       1985-01-07
4       1985-01-07
           ...    
24955   1995-06-29
24956   1995-06-29
24957   1995-09-23
24958   1996-06-25
24959   1996-06-25
Name: dob, Length: 24960, dtype: datetime64[ns]

In [248]:
#Calculate age at race in days (timedelta)
results_with_dob_date['ageAtRace'] = results_with_dob_date['date'] - results_with_dob_date['dob']

In [249]:
#Calculate how many days in f1 in days (timedelta)
date_of_debut = results_with_dob_date.groupby('driverId')['date'].min().to_frame()
date_of_debut = date_of_debut.merge(drivers, on = 'driverId', how = 'left').rename(columns={'date':'date_of_debut'})
date_of_debut = date_of_debut[['driverId','date_of_debut']]
date_of_debut['date_of_debut']
df = results_with_dob_date.merge(date_of_debut, on = 'driverId', how = 'left')
df['ageAtDebut'] = df['date_of_debut'] - df['dob']
df['yearsExperience'] = df['date'] - df['date_of_debut']


In [250]:
#Drop columns
df = df.drop(columns=['number_y', 'number_x', 'code', 'forename', 'surname', 'dob', 'url_x', 'time_y', 'url_y'])

In [251]:
#Rename Columns
df = df.rename(columns={"grid": "gridStart", "name": "circuitName", "date_of_debut" : "dateOfDebut", "points": "pointsGained"})

In [252]:
#Add is racing at home feature
df['racingAtHome'] = df.apply(lambda row: row.nationality in row.circuitName, axis=1)

In [253]:
#COnvert ages from days to years
df['ageAtRace'] = df.apply(lambda row: row.ageAtRace / pd.Timedelta('365 days'), axis=1)
df['ageAtDebut'] = df.apply(lambda row: row.ageAtDebut / pd.Timedelta('365 days'), axis=1)
df['yearsExperience'] = df.apply(lambda row: row.yearsExperience / pd.Timedelta('365 days'), axis=1)

In [254]:
#Add starts in front 2 places on the grid
df['startsFrontRow'] = df['gridStart'] <= 2

In [255]:
#Merge with driver standings
driver_standings = driver_standings.rename(columns={"points": "driverStandingsPoints", "position": "driverStandingsPosition", "wins" : "driverStandingsWins"})
driver_standings = driver_standings.drop(columns=['positionText'])
df = df.merge(driver_standings, on = ['raceId','driverId'], how = 'left')

In [256]:
#Get driver standings information before race
df['driverStandingsPoints'] = df.groupby(['year','driverId'])['driverStandingsPoints'].shift(fill_value=0)
df['driverStandingsPosition'] = df.groupby(['year','driverId'])['driverStandingsPosition'].shift(fill_value=0)
df['driverStandingsWins'] = df.groupby(['year','driverId'])['driverStandingsWins'].shift(fill_value=0)

In [257]:
#Merge with constructor standings
constructor_standings = constructor_standings.rename(columns={"points": "constructorStandingsPoints", "position": "constructorStandingsPosition", "wins" : "constructorStandingsWins"})
constructor_standings = constructor_standings.drop(columns=['positionText'])
df = df.merge(constructor_standings, on = ['raceId','constructorId'], how = 'left')

In [260]:
#Get constructor standings information before race
df['constructorStandingsPoints'] = df.groupby(['year','driverId'])['constructorStandingsPoints'].shift(fill_value=0)
df['constructorStandingsPosition'] = df.groupby(['year','driverId'])['constructorStandingsPosition'].shift(fill_value=0)
df['constructorStandingsWins'] = df.groupby(['year','driverId'])['constructorStandingsWins'].shift(fill_value=0)

In [261]:
df.head()

,resultId,raceId,driverId,constructorId,gridStart,position,positionText,positionOrder,pointsGained,laps,...,racingAtHome,startsFrontRow,driverStandingsId,driverStandingsPoints,driverStandingsPosition,driverStandingsWins,constructorStandingsId,constructorStandingsPoints,constructorStandingsPosition,constructorStandingsWins
0,1,18,1,1,1,1,1,1,10.0,58,...,False,True,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,27,19,1,1,9,5,5,5,4.0,56,...,False,False,9.0,10.0,1.0,1.0,7.0,14.0,1.0,1.0
2,57,20,1,1,3,13,13,13,0.0,56,...,False,False,27.0,14.0,1.0,1.0,18.0,24.0,1.0,1.0
3,69,21,1,1,5,3,3,3,6.0,66,...,False,False,48.0,14.0,3.0,1.0,29.0,28.0,3.0,1.0
4,90,22,1,1,3,2,2,2,8.0,58,...,False,False,69.0,20.0,2.0,1.0,40.0,34.0,3.0,1.0
